In [1]:
!pip install torch==2.2.0 torchaudio torchsummary torchtext torchvision

INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchtext to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 92.1 MB/s e

In [2]:
!pip install numpy==1.24.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 88.7 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.4.15 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
bayesian-optimization 2.0.1 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 18.1.0 which is incompatible.
featuretools 1.31.0 requires numpy>=1.25.0, but you have numpy 1.24.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 18.1.0 which is incompatible.
pandas-gbq 0.23.1 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1

In [3]:
!pip install spacy

In [4]:
!pip install timm

In [5]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random as rand
import os

from PIL import Image
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms
from torchtext.vocab import build_vocab_from_iterator

import spacy
import timm

In [7]:
def set_seed(seed):
  rand.seed(seed)
  np.random.seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

seed = 59
set_seed(59)
print("Done!")

Done!


In [9]:
data = open('/kaggle/input/dataset/vaq2.0.TrainImages.txt', 'r')
lines = data.readlines()
print(lines[:5])  # Print the first 5 lines

['COCO_val2014_000000393225.jpg#0\tIs this a creamy soup ? no\n', 'COCO_val2014_000000393243.jpg#0\tIs this person wearing a tie ? no\n', 'COCO_val2014_000000262197.jpg#0\tIs this a hospital ? yes\n', 'COCO_val2014_000000393277.jpg#0\tAre there any tour buses ? no\n', 'COCO_val2014_000000393277.jpg#1\tIs this a one way street ? no\n']


In [11]:
# Load train data
train_data = []
train_path = '/kaggle/input/dataset/vaq2.0.TrainImages.txt'
with open(train_path, 'r') as f:
    for i, line in enumerate(f.readlines()):
        
        full_sentence = line.split('\t')
        if (i < 3):
            print("Full sentence: ", full_sentence)
        
        img_path = full_sentence[0][:-2]
        if (i < 3):
            print("Image Path: ", img_path)
        
        qa = full_sentence[1].split('?')
        
        question = qa[0]
        if (i < 3):
            print("Question: ", question)

        # Error handling in case
        if len(qa) == 3:
            answer = qa[2]
        else:
            answer = qa[1]
        
        # Remove any trailing newline characters or extra spaces from the answer
        answer = answer.strip()
        
        if (i < 3):
            print("Answer: ", answer)
            
        if (i < 3):
            print(" ")
            
        data_sample = {
            'Image Path': img_path,
            'Question': question + '?',
            'Answer': answer  # No trailing newline
        }
        train_data.append(data_sample)

Full sentence:  ['COCO_val2014_000000393225.jpg#0', 'Is this a creamy soup ? no\n']
Image Path:  COCO_val2014_000000393225.jpg
Question:  Is this a creamy soup 
Answer:  no
 
Full sentence:  ['COCO_val2014_000000393243.jpg#0', 'Is this person wearing a tie ? no\n']
Image Path:  COCO_val2014_000000393243.jpg
Question:  Is this person wearing a tie 
Answer:  no
 
Full sentence:  ['COCO_val2014_000000262197.jpg#0', 'Is this a hospital ? yes\n']
Image Path:  COCO_val2014_000000262197.jpg
Question:  Is this a hospital 
Answer:  yes
 


In [12]:
# Load val data
val_data = []
val_path = '/kaggle/input/dataset/vaq2.0.DevImages.txt'
with open(val_path, 'r') as f:
    for i, line in enumerate(f.readlines()):
        
        full_sentence = line.split('\t')
        if (i < 3):
            print("Full sentence: ", full_sentence)
        
        img_path = full_sentence[0][:-2]
        if (i < 3):
            print("Image Path: ", img_path)
        
        qa = full_sentence[1].split('?')
        
        question = qa[0]
        if (i < 3):
            print("Question: ", question)

        # Error handling in case
        if len(qa) == 3:
            answer = qa[2]
        else:
            answer = qa[1]
        
        # Remove any trailing newline characters or extra spaces from the answer
        answer = answer.strip()
        
        if (i < 3):
            print("Answer: ", answer)
            
        if (i < 3):
            print(" ")
            
        data_sample = {
            'Image Path': img_path,
            'Question': question + '?',
            'Answer': answer  # No trailing newline
        }
        val_data.append(data_sample)

Full sentence:  ['COCO_val2014_000000262175.jpg#0', 'Is this a designer tie ? no\n']
Image Path:  COCO_val2014_000000262175.jpg
Question:  Is this a designer tie 
Answer:  no
 
Full sentence:  ['COCO_val2014_000000393284.jpg#1', 'Is this man snowboarding ? yes\n']
Image Path:  COCO_val2014_000000393284.jpg
Question:  Is this man snowboarding 
Answer:  yes
 
Full sentence:  ['COCO_val2014_000000000133.jpg#0', 'Is this a child room ? yes\n']
Image Path:  COCO_val2014_000000000133.jpg
Question:  Is this a child room 
Answer:  yes
 


In [13]:
# Load train data
test_data = []
test_path = '/kaggle/input/dataset/vaq2.0.TestImages.txt'
with open(test_path, 'r') as f:
    for i, line in enumerate(f.readlines()):
        
        full_sentence = line.split('\t')
        if (i < 3):
            print("Full sentence: ", full_sentence)
        
        img_path = full_sentence[0][:-2]
        if (i < 3):
            print("Image Path: ", img_path)
        
        qa = full_sentence[1].split('?')
        
        question = qa[0]
        if (i < 3):
            print("Question: ", question)

        # Error handling in case
        if len(qa) == 3:
            answer = qa[2]
        else:
            answer = qa[1]
        
        # Remove any trailing newline characters or extra spaces from the answer
        answer = answer.strip()
        
        if (i < 3):
            print("Answer: ", answer)
            
        if (i < 3):
            print(" ")
            
        data_sample = {
            'Image Path': img_path,
            'Question': question + '?',
            'Answer': answer  # No trailing newline
        }
        test_data.append(data_sample)

Full sentence:  ['COCO_val2014_000000262162.jpg#0', 'Are there any boxes in the room ? no\n']
Image Path:  COCO_val2014_000000262162.jpg
Question:  Are there any boxes in the room 
Answer:  no
 
Full sentence:  ['COCO_val2014_000000393277.jpg#0', 'Are there any any tour buses ? no\n']
Image Path:  COCO_val2014_000000393277.jpg
Question:  Are there any any tour buses 
Answer:  no
 
Full sentence:  ['COCO_val2014_000000393284.jpg#2', 'Is this person flying ? no\n']
Image Path:  COCO_val2014_000000393284.jpg
Question:  Is this person flying 
Answer:  no
 


In [14]:
eng = spacy.load('en_core_web_sm')

def get_tokens(data_iter):
    """
    Input: data (dictionary)
    """
    for sample in data_iter:
        question = sample['Question']
        yield[token.text for token 
              in eng.tokenizer(question)]

# Build vocab from train set
vocab = build_vocab_from_iterator(
    get_tokens(train_data),
    min_freq = 2, # Appears atleast 2 times
    specials = ['<pad>', '<sos>', '<eos>', '<unk>'],
    special_first = True
)
vocab.set_default_index (vocab['<unk>'])
print("Done!")

Done!


In [15]:
# Get all classes
classes = set([sample['Answer'] for sample in train_data])

label2idx = {
    cls_name: idx for idx, cls_name in enumerate(classes)
}
print(label2idx)
print("Keys: ", label2idx.keys())
print("Values: ", label2idx.values())

print("")

idx2label = {
    idx: cls_name for idx, cls_name in enumerate(classes)
}
print(idx2label)
print("Keys: ", idx2label.keys())
print("Values: ", idx2label.values())

{'no': 0, 'yes': 1}
Keys:  dict_keys(['no', 'yes'])
Values:  dict_values([0, 1])

{0: 'no', 1: 'yes'}
Keys:  dict_keys([0, 1])
Values:  dict_values(['no', 'yes'])


In [16]:
def tokenize(question, max_seq_len):
    tokens = [token.text for token in eng.tokenizer(question)] # Tokenizes questions
    seq = [vocab[token] for token in tokens]
    if len(seq) < max_seq_len:
        seq += [vocab['<pad>']] * (max_seq_len - len(seq)) # Add padding
    else:
        seq = seq[:max_seq_len] # Take until maxed length reached
    
    return seq
print("Done!")

Done!


In [17]:
class VQA_dataset(Dataset):
    def __init__(self, data, label2idx, max_seq_len = 20, transform=None, img_dir = '/kaggle/input/dataset/val2014-resised'):
        self.transform = transform
        self.data = data
        self.max_len = max_seq_len
        self.img_dir = img_dir
        self.label2idx = label2idx

    # Returns: Number of samples
    def __len__(self):
        return len(self.data)
        
    # Returns: Transformed Image, tensorized Question & Label
    def __getitem__(self, idx):
        
        # /kaggle/input/dataset/val2014-resised/Image Path
        img_path = os.path.join(self.img_dir,
                               self.data[idx]['Image Path'])
        
        # Open image and convert to RGB
        img = Image.open(img_path).convert('RGB') 
        

        if self.transform:
            img = self.transform(img)
            question = self.data[idx]['Question'] # Get question at index
            question = tokenize(question, self.max_len) # Tokenizes question
            question = torch.tensor(question, dtype=torch.long) # Converts to tensor as long type

            label = self.data[idx]['Answer'] # Get answer/label
            label = label2idx[label] # [0, 1]
            label = torch.tensor(label, dtype = torch.long) # Converts to tensor as long type

        return img, question, label
print("Done!")

Done!


In [18]:
# Dictionary that stores all transformation methods for train & val
data_transform = {
    'train': transforms.Compose(
        [
            transforms.Resize(size = (224, 224)),
            transforms.CenterCrop(size = 180),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(brightness = 0.1,
                                  contrast = 0.1,
                                  saturation = 0.1),
            transforms.RandomHorizontalFlip(),
            transforms.GaussianBlur(3),
            transforms.ToTensor(),
            transforms.Normalize(
                (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)
            )
        ]
    ),

    'val': transforms.Compose(
        [
        transforms.Resize(size = (224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)
            )
        ]
    )
}
print("Done!")

Done!


In [19]:
train_set = VQA_dataset(
    train_data,
    label2idx = label2idx,
    transform = data_transform['train'] # Get all transformation for train
)

val_set = VQA_dataset(
    val_data,
    label2idx = label2idx,
    transform = data_transform['val'] # Get all transformation for val
)

test_set = VQA_dataset(
    test_data,
    label2idx = label2idx,
    transform = data_transform['val'] # Same transformation as val
)

print(train_set)
print(val_set)
print(test_set)

In [20]:
train_batch = 256
test_batch = 32

train_loader = DataLoader(
    train_set,
    batch_size = train_batch,
    shuffle = True
)

val_loader = DataLoader(
    val_set,
    batch_size = test_batch,
    shuffle = False
)

test_loader = DataLoader(
    test_set,
    batch_size = test_batch,
    shuffle = False
)
print(train_loader)
print(val_loader)
print(test_loader)

In [21]:
class VQA_model(nn.Module):
    def __init__(self, num_class, img_model_name, embed_dim,
                num_layers = 2,
                hidden_size = 256,
                dropout_prob = 0.2):
        super(VQA_model, self).__init__()

        self.img_encoder = timm.create_model(
            img_model_name,
            pretrained = True,
            num_classes = hidden_size
        )
        for param in self.img_encoder.parameters():
            param.requires_grad = True


        self.embed = nn.Embedding(len(vocab), embed_dim)

        self.lstm1 = nn.LSTM(
            input_size = embed_dim,
            hidden_size = hidden_size,
            num_layers = num_layers,
            batch_first = True,
            bidirectional = True,
            dropout = dropout_prob
        )
        
        """
            2 Layers LSTM => output 2 * Hidden Size
            Concatenated encoded image => output Hidden Size
            => Total nodes  = Hidden Size + 2 * Hidden Size = 3 * Hidden Size
        """
        
        self.fc1 = nn.Linear(hidden_size * 3, hidden_size)
        self.dropout = nn.Dropout(dropout_prob)
        self.gelu = nn.GELU()
        self.fc2 = nn.Linear(hidden_size, num_class)

    def forward(self, img, text):
        img = self.img_encoder(img)

        text = self.embed(text)
        lstm_out, _ = self.lstm1(text)  # Get the tuple from LSTM
        lstm_out = lstm_out[:, -1, :]   # Take the last hidden state from the output tensor


        Concat = torch.cat((img, lstm_out), dim = 1) # Concat Encoded img & Encoded text
        x = self.fc1(Concat)
        x = self.gelu(x)
        x = self.dropout(x)
        x = self.fc2(x)

        return x
print("Done!")

Done!


In [22]:
num_class = len(classes)
img_model_name = 'resnet18'
hidden_size = 256
num_layers = 2
embed_dim = 128
dropout_prob = 0.2

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = VQA_model(
    num_class = num_class,
    img_model_name = img_model_name,
    embed_dim = embed_dim,
    hidden_size = hidden_size,
    num_layers = num_layers,
    dropout_prob = dropout_prob
).to(device)

print("Done!")

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Done!


In [23]:
def eval(model, val_set, criterion, device):
    model.eval()
    corr = 0
    total = 0
    losses = []

    with torch.no_grad():
        for img, question, labels in val_set:
            
            img = img.to(device)
            question = question.to(device)
            labels = labels.to(device)

            outputs = model(img, question)
            loss = criterion(outputs, labels)

            losses.append(loss.item())
            _, pred = torch.max(outputs.data, 1) # Index of highest row
            total += labels.size(0)
            corr += (pred == labels).sum().item()
    loss = sum(losses)/len(losses)
    acc = corr/total

    return loss, acc
print("Done!")

Done!


In [36]:
import time
def train(model, train_data, val_data, criterion, optim, scheduler, device, epochs):
    train_loss = []
    val_loss = []
    times = []
    for i in range(epochs):
        print("-" * 59)
        print(f'Starting Epoch {i + 1}...')
        epoch_start_time = time.time()
        batch_loss = []
        model.train()
        
        for idx, (image, questions, labels) in enumerate(train_data):
            image = image.to(device)
            questions = questions.to(device)
            labels = labels.to(device)

            optim.zero_grad()

            out = model(image, questions)
            loss = criterion(out, labels)
            loss.backward()
            optim.step()
            batch_loss.append(loss.item())
            
            print(f"| Epoch: {i + 1} | {idx + 1}/{len(train_data)} Batches | Train Loss: {loss.item():.4f} |")

        train_loss1 = sum(batch_loss) / len(batch_loss)
        train_loss.append(train_loss1)
        val_loss1, val_acc = eval(
            model, val_data, criterion, device
        )
        val_loss.append(val_loss1)
        times.append(time.time() - epoch_start_time)
        
        print(f"| Epoch: {i + 1}/{epochs:3d} | Train Loss: {train_loss1:.4f} | Val Loss: {val_loss1:.4f} | Val Accuracy: {val_acc:.2f} | Time: {(time.time() - epoch_start_time):.2f}s |")
        print(f"Epoch {i + 1} was ran successfully")
        print("-"*59)
        scheduler.step()
    return train_loss, val_loss
print("Done!")


Done!


In [25]:
lr = 1e-3
epochs = 50

scheduler_step_size = epochs*0.8
criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(
    model.parameters(),
    lr = lr
)

scheduler = torch.optim.lr_scheduler.StepLR(
    optim,
    step_size = scheduler_step_size,
    gamma = 0.1
)
print("Done!")

Done!


In [37]:
train_loss, val_loss = train(model,
                            train_loader,
                            val_loader,
                            criterion,
                            optim,
                            scheduler,
                            device,
                            epochs)
print("Done!")

Starting Epoch 1...
| Epoch: 1 | 1/31 Batches | Train Loss: 0.2466 |
| Epoch: 1 | 2/31 Batches | Train Loss: 0.3287 |
| Epoch: 1 | 3/31 Batches | Train Loss: 0.3551 |
| Epoch: 1 | 4/31 Batches | Train Loss: 0.3529 |
| Epoch: 1 | 5/31 Batches | Train Loss: 0.3293 |
| Epoch: 1 | 6/31 Batches | Train Loss: 0.2863 |
| Epoch: 1 | 7/31 Batches | Train Loss: 0.2888 |
| Epoch: 1 | 8/31 Batches | Train Loss: 0.2729 |
| Epoch: 1 | 9/31 Batches | Train Loss: 0.3377 |
| Epoch: 1 | 10/31 Batches | Train Loss: 0.3256 |
| Epoch: 1 | 11/31 Batches | Train Loss: 0.3309 |
| Epoch: 1 | 12/31 Batches | Train Loss: 0.3405 |
| Epoch: 1 | 13/31 Batches | Train Loss: 0.3306 |
| Epoch: 1 | 14/31 Batches | Train Loss: 0.3318 |
| Epoch: 1 | 15/31 Batches | Train Loss: 0.3060 |
| Epoch: 1 | 16/31 Batches | Train Loss: 0.3392 |
| Epoch: 1 | 17/31 Batches | Train Loss: 0.3017 |
| Epoch: 1 | 18/31 Batches | Train Loss: 0.4373 |
| Epoch: 1 | 19/31 Batches | Train Loss: 0.3659 |
| Epoch: 1 | 20/31 Batches | Train Loss

In [39]:
val_loss, val_acc = eval(model,
                        val_loader,
                        criterion,
                        device)

test_loss, test_acc = eval(
    model,
    test_loader,
    criterion,
    device
)
print("Done!")

Done!


In [42]:
print(f"Val Accuracy: {val_acc}")
print(f"Test Accuracy: {test_acc}")
print("Done!")

Val Accuracy: 0.6362704918032787
Test Accuracy: 0.6572700296735905
Done!
